In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("silkroad2")

create_database('blp_dataset')
create_table('products_tmp')

IMG_FINAL_FOLDER = r"D:\images\silkroad2"

In [ ]:
files_read = []
file_error_image = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\items")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            
            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                desc_content = page_content.find("div", attrs={"class": "container container_large"})

                if desc_content is not None:
                    description = get_content(desc_content)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("h2")
                
                if raw_title is not None:
                    title = get_content(raw_title)

                ## GET SELLER
                seller = ""
                seller_content = page_content.find("h3")

                if seller_content is not None:
                    raw_seller = seller_content.find("a")
                    
                    if raw_seller is not None:
                        seller = get_content(raw_seller)
                
                ## GET PRICE
                price = ""
                price_content = page_content.find("div", attrs={"class": "price_big"})

                if price_content is not None:
                    price = get_content(price_content)

                product_id = 0

                if description != "" or title != "":
                    try:
                        # Insert product into database
                        query = "INSERT INTO products_tmp (name, description, market_name, seller_name, price, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
                        values = (title, description, "SilkRoad 2", seller, price, "t")
                    
                        cursor.execute(query, values)
                        conn.commit()

                        product_id = cursor.lastrowid
                    except:
                        print("Error on insert")
                    
                    if product_id > 0:
                        ## GET IMAGE
                        raw_name = page_content.find_all("link", href=lambda href: href and "image" in href)
                        img_folder = Path(f"{folder_image}\images")

                        if raw_name is not None and raw_name:
                            href = raw_name[0]
                            
                            if href and href is not None:
                                href = href["href"]
                                keyword = re.findall(r'\d+', href)

                                if keyword:
                                    keyword = keyword[0]

                                    if keyword is not None:
                                        c_img = 0
                                        num_img = 0

                                        for fname in os.listdir(img_folder):
                                            if keyword in fname:
                                                if fname not in files_read:
                                                    files_read.append(fname)
                                                    c_img = c_img + 1                                    
                                                    css = open(Path(f"{img_folder}\{fname}")).read()
                                                    sheet = cssutils.parseString(css)

                                                    data_urls = list(cssutils.getUrls(sheet))

                                                    for img_data in data_urls:
                                                        if num_img == 0:
                                                            img_filename = (Path(f"{IMG_FINAL_FOLDER}\{product_id}.jpg"))
                                                        else:
                                                            img_filename = (Path(f"{IMG_FINAL_FOLDER}\{product_id}_{num_img}.jpg"))

                                                        try:
                                                            data = "" + img_data.replace("data:image/jpeg;base64,", "")
                                                            imgdata = base64.b64decode(data)

                                                            with open(img_filename, 'wb') as f:
                                                                f.write(imgdata)
                                                                f.close
                                                            num_img = num_img + 1
                                                        except:
                                                            print(f"Image {img_filename} error")
                                                            file_error_image.append(f"{file} - {fo}")
                                                    if num_img == 0:
                                                        update_image(product_id, "no_image")
                                                        file_error_image.append(f"{file} - {fo}")
                                                else:
                                                    print("File already visited")
                                                    update_image(product_id, "no_image")
                                        if c_img == 0:
                                            print(f"Image {product_id} error getting name")
                                            update_image(product_id, "no_image")
                                            file_error_image.append(f"{file} - {fo}")
                                    else:
                                        print(f"Image {product_id} error getting name")
                                        update_image(product_id, "no_image")
                                        file_error_image.append(f"{file} - {fo}")